# 연습문제3 - 분산분석

통계분석의 주요 검정 기법

| 기법 | 설명 |
|---|---|
| 독립표본 t검정 | 독립된 두 그룹의 평균비교(정규분포 가정이 가능할 때) |
| 대응표본 t검정 | 쌍을 이룬 두 그룹의 평균비교(정규분포 가정이 가능할 때) |
| 분산분석 | 세 그룹의 평균 비교 (정규분포 가정이 가능할 때) |
| 상관분석 | 두 수치형 변수 사이의 상관관계 유무와 강도 계산 |
| 회귀분석 | 종속변수와 독립변수의 관계를 모델링 |

In [14]:
from pandas import read_excel

# 분산분석을 위한 라이브러리
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import shapiro, normaltest, ks_2samp, bartlett, fligner, levene, f_oneway, chi2_contingency

# 사후분석을 위한 라이브러리
from statsmodels.sandbox.stats.multicomp import MultiComparison
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_ind

# helper 참조
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
from helper import normality_test, equal_variance_test, independence_test, all_test

## 문제1
유아들을 대상으로 세 가지 읽는 방법을 비교,실험하여 다음과 같은 독서평가 점수자료를 얻었다.

세 방법에 통계적으로 유의한 차이가 있는지 검정하라.

In [15]:
df = read_excel("https://data.hossam.kr/E02/reading.xlsx")
df

,독서방법,점수
0,B,6
1,B,9
2,B,2
3,B,16
4,D,7
5,D,7
6,D,12
7,D,10
8,D,16
9,S,11


In [16]:
df.dtypes

독서방법    object
점수       int64
dtype: object

### 전처리

In [17]:
# 데이터 복사해 원본 유지
df1 = df.copy()
# df1['독서방법'] = df1['독서방법'].astype('category')
# value 값 변경
df1['독서방법'] = df1['독서방법'].map({'B': 0, 'D': 1, 'S': 2})
df1

,독서방법,점수
0,0,6
1,0,9
2,0,2
3,0,16
4,1,7
5,1,7
6,1,12
7,1,10
8,1,16
9,2,11


### 일원분산분석

helper.py 모듈로 정규성, 등분산성, 독립성 확인

In [18]:
all_test(df1['독서방법'], df1['점수'])

c:\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


statistic   p-value  result
condition test       field                                  
정규성       shapiro    독서방법         0.810449  0.006740   False
                     점수           0.925500  0.263591    True
          normaltest 독서방법         3.635337  0.162404    True
                     점수           1.019042  0.600783    True
          ks_2samp   독서방법 vs 점수   0.928571  0.000001   False
                     점수 vs 독서방법   0.928571  0.000001   False
등분산성      Bartlett   독서방법 vs 점수  23.394518  0.000001   False
          Fligner    독서방법 vs 점수   4.592595  0.032110   False
          Levene     독서방법 vs 점수   7.088392  0.013132   False
독립성       Chi2       독서방법 vs 점수  11.514766  0.567776    True

#### 일원분산분석 결과
문제1의 정규성을 통해 독서방법과 점수의 상관관계는 약 `0.929`로 강한 상관관계를 가지고 `p-value(유의수준)`값이 약 $1\mu$로 0.05보다 작기 때문에 `귀무가설을 기각`하고 `대립가설을 채택`한다. 결론적으로 독서방법과 점수는 서로 관계가 있고 볼 수 있다.

### 분산분석 수행

In [19]:
# 결과보고에 참고할 유의미한 데이터
model = ols('점수 ~ C(독서방법)', data=df1).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(독서방법),2.0,26.464286,13.232143,0.779403,0.48244
Residual,11.0,186.750000,16.977273,NaN,NaN


### 결과보고
> 독서방법에 따른 성적의 평균 차이는 무의미하였다($F(2, 11) = 0.779, p > 0.05$) (p-value = 0.48244).

## 문제2

효소 성분의 비료가 효소 함량이 증가함에 따라 토마토 모종 포기의 성장에 어떤 영향을 주는지 실험연구를 하여 다음 자료를 얻었다.

분산분석을 하여 효소 함량에 따라 토마토 모종 성장에 차이가 나는지 검정하라.

In [20]:
df = read_excel("https://data.hossam.kr/E02/tomato.xlsx")
df

,효소함량,모종성장
0,0,10.8
1,0,9.1
2,0,13.5
3,0,9.2
4,1000,11.1
5,1000,11.2
6,1000,8.2
7,1000,11.3
8,5000,5.4
9,5000,4.6


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   효소함량    16 non-null     int64  
 1   모종성장    16 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 384.0 bytes


In [22]:
df['효소함량']=df['효소함량'].map({0:1, 1000:2, 5000:3, 10000:4})
df

,효소함량,모종성장
0,1,10.8
1,1,9.1
2,1,13.5
3,1,9.2
4,2,11.1
5,2,11.2
6,2,8.2
7,2,11.3
8,3,5.4
9,3,4.6


In [24]:
# 결과보고에 참고할 유의미한 데이터
model = ols('모종성장 ~ C(효소함량)', data=df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(효소함량),3.0,101.1275,33.709167,12.085748,0.000615
Residual,12.0,33.4700,2.789167,NaN,NaN


## 문제3

이원분산분석